In [1]:
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal, XSD, RDF, RDFS
from rdflib.namespace import RDF, RDFS, OWL

# Создаем граф RDF
g = Graph()

# Определяем пространства имен
BASE = Namespace("http://example.org/skyrim#")
g.bind("base", BASE)

# Определяем классы
classes = {
    "Book": BASE.Book,
    "Character": BASE.Character,
    "Weapon": BASE.Weapon,
    "Skill": BASE.Skill,
    "Race": BASE.Race,
    "Faction": BASE.Faction,
    "Location": BASE.Location
}

for cls in classes.values():
    g.add((cls, RDF.type, RDFS.Class))

# Определяем свойства
data_properties = {
    "relatedSkill": BASE.relatedSkill,
    "requiresSkill": BASE.requiresSkill,
    "madeOf": BASE.madeOf,
    "cost": BASE.cost,
    "damage": BASE.damage,
    "weight": BASE.weight,
}
obj_properties = {
    "hasSkill": BASE.hasSkill,
    "usesWeapon": BASE.usesWeapon,
    "isMemberOf": BASE.isMemberOf,
    "locatedAt": BASE.locatedAt,    
    "authoredBy": BASE.authoredBy,
    "isFriendOf": BASE.isFriendOf,
    "hasRace": BASE.hasRace
    
}

for prop in data_properties.values():
    g.add((prop, RDF.type, OWL.DatatypeProperty))
for prop in obj_properties.values():
    g.add((prop, RDF.type, OWL.ObjectProperty))  
    
# Указываем характеристики для свойств
g.add((BASE.cost, RDFS.domain, BASE.Weapon))
g.add((BASE.cost, RDFS.range, XSD.string))

g.add((BASE.weight, RDFS.domain, BASE.Weapon))
g.add((BASE.weight, RDFS.range, XSD.string))

g.add((BASE.damage, RDFS.domain, BASE.Weapon))
g.add((BASE.damage, RDFS.range, XSD.string))

g.add((BASE.madeOf, RDFS.domain, BASE.Weapon))
g.add((BASE.madeOf, RDFS.range, XSD.string))

# relatedSkill
g.add((BASE.relatedSkill, RDFS.domain, BASE.Book))
g.add((BASE.relatedSkill, RDFS.range, XSD.boolean))

#requiresSkill
g.add((BASE.requiresSkill, RDFS.domain, BASE.Weapon))
g.add((BASE.requiresSkill, RDFS.range, XSD.string))

# hasSkill - Функциональное свойство, Domain - Character, Range - Skill
g.add((BASE.hasSkill, RDF.type, OWL.FunctionalProperty))
g.add((BASE.hasSkill, RDFS.domain, BASE.Character))
g.add((BASE.hasSkill, RDFS.range, BASE.Skill))

# isMemberOf - не симметричное свойство, Domain - Character, Range - Faction
g.add((BASE.isMemberOf, RDFS.domain, BASE.Character))
g.add((BASE.isMemberOf, RDFS.range, BASE.Faction))

# usesWeapon - не транзитивное свойство, Domain - Character, Range - Weapon
g.add((BASE.usesWeapon, RDFS.domain, BASE.Character))
g.add((BASE.usesWeapon, RDFS.range, BASE.Weapon))

# locatedAt - Функциональное свойство, Domain - Character, Range - Location
g.add((BASE.locatedAt, RDF.type, OWL.FunctionalProperty))
g.add((BASE.locatedAt, RDFS.domain, BASE.Character))
g.add((BASE.locatedAt, RDFS.range, BASE.Location))

# authoredBy - не симметричное свойство, Domain - Book, Range - Character
g.add((BASE.authoredBy, RDFS.domain, BASE.Book))
g.add((BASE.authoredBy, RDFS.range, BASE.Character))

# Добавляем свойство isFriendOf как транзитивное
g.add((BASE.isFriendOf, RDF.type, OWL.TransitiveProperty))  # Указание, что это транзитивное свойство
g.add((BASE.isFriendOf, RDFS.domain, BASE.Character))
g.add((BASE.isFriendOf, RDFS.range, BASE.Character))


# Загрузка данных из CSV
books_df = pd.read_csv("data/skyrim_books.csv")
characters_df = pd.read_csv("data/skyrim_Named_Characters.csv")
weapons_df = pd.read_csv("data/Skyrim_Weapons.csv")


# Добавляем книги в граф
for _, row in books_df.iterrows():
    book_uri = URIRef(BASE[row["Title"].replace(" ", "_")])
    g.add((book_uri, RDF.type, classes["Book"]))
    g.add((book_uri, BASE.cost, Literal(row["Cost"])))
    g.add((book_uri, BASE.description, Literal(row["Description"])))
    
    if pd.notna(row["Author"]):
        author_uri = URIRef(BASE[str(row["Author"]).replace(" ", "_")])
        # g.add((author_uri, RDF.type, OWL.ObjectProperty))
        g.add((author_uri, RDF.type, classes["Character"]))
        g.add((book_uri, obj_properties["authoredBy"], author_uri))
    
    if row["Skill"] == 1:  # Если Skill равно True, то книга дает навык
        g.add((book_uri, data_properties["relatedSkill"], Literal(True)))
    else:
        g.add((book_uri, data_properties["relatedSkill"], Literal(False)))
    
# Добавляем персонажей в граф
for _, row in characters_df.iterrows():
    character_uri = URIRef(BASE[row["Name"].replace(" ", "_")])
    # g.add((character_uri, RDF.type, OWL.ObjectProperty))
    g.add((character_uri, RDF.type, classes["Character"]))
    
    if pd.notna(row["Race"]):
        race_uri = URIRef(BASE[row["Race"].replace(" ", "_")])
        g.add((race_uri, RDF.type, classes["Race"]))
        g.add((character_uri, BASE.hasRace, race_uri))
    
    if pd.notna(row["Location"]):
        location_uri = URIRef(BASE[row["Location"].replace(" ", "_")])
        g.add((location_uri, RDF.type, classes["Location"]))
        g.add((character_uri, obj_properties["locatedAt"], location_uri))
    
    if pd.notna(row["Faction(s)"]):
        factions = row["Faction(s)"].split("; ")
        for faction in factions:
            faction_uri = URIRef(BASE[faction.replace(" ", "_")])
            g.add((faction_uri, RDF.type, classes["Faction"]))
            g.add((character_uri, BASE.isMemberOf, faction_uri))
            
        for faction in factions:
            faction_uri = URIRef(BASE[faction.replace(" ", "_")])
            
            # # Ищем всех других персонажей, которые принадлежат этой же фракции
            # for _, other_row in characters_df.iterrows():
            #     if other_row["Name"] != row["Name"] and pd.notna(other_row["Faction(s)"]):
            #         other_factions = other_row["Faction(s)"].split("; ")
            #         if faction in other_factions:
            #             other_character_uri = URIRef(BASE[other_row["Name"].replace(" ", "_")])
            #             # Добавляем связь "isFriendOf"
            #             g.add((character_uri, BASE.isFriendOf, other_character_uri))
            #             g.add((other_character_uri, BASE.isFriendOf, character_uri))
    
    if pd.notna(row["Primary Skills"]):
        skills = row["Primary Skills"].split(", ")
        for skill in skills:
            skill_uri = URIRef(BASE[skill.replace(" ", "_")])
            g.add((skill_uri, RDF.type, classes["Skill"]))
            g.add((character_uri, obj_properties["hasSkill"], skill_uri))

# Добавляем оружие в граф
for _, row in weapons_df.iterrows():
    weapon_uri = URIRef(BASE[row["Name"].replace(" ", "_")])
    g.add((weapon_uri, RDF.type, classes["Weapon"]))
    g.add((weapon_uri, BASE.cost, Literal(row["Gold"])))
    g.add((weapon_uri, BASE.damage, Literal(row["Damage"])))
    g.add((weapon_uri, BASE.weight, Literal(row["Weight"])))
    # Свойство madeOf (материал оружия)
    if pd.notna(row["Upgrade"]):
        # material_uri = URIRef(BASE[row["Type"].replace(" ", "_")])
        # g.add((material_uri, RDF.type, RDFS.Literal))
        g.add((weapon_uri, BASE.madeOf, Literal(row["Upgrade"])))
        
    # Свойство requiresSkill (необходимый навык для использования оружия)
    if pd.notna(row["Category"]):
        # skill_uri = URIRef(BASE[row["Category"].replace(" ", "_")])
        # g.add((skill_uri, RDF.type, classes["Skill"]))
        g.add((weapon_uri, data_properties["requiresSkill"], Literal(row["Category"])))
    
# Сохраняем граф в формате Turtle для импорта в Protege
g.serialize("skyrim_knowledge_graph.owl", format="xml")


<Graph identifier=N7711b7bd13c6408380e655bf7a9802e9 (<class 'rdflib.graph.Graph'>)>